## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import gkey

# Configure gmaps API key
gmaps.configure(api_key=gkey)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rundu,NaN,-17.9333,19.7667,62.74,27,0,4.83,clear sky
1,1,Filiatra,GR,37.1500,21.5833,72.75,80,9,15.10,clear sky
2,2,Vaini,TO,-21.2000,-175.2000,68.16,88,20,6.91,few clouds
3,3,Payo,ID,-3.7508,103.6372,73.72,96,100,1.97,moderate rain
4,4,Maarianhamina,AX,60.0973,19.9348,46.26,93,90,9.22,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip (F)?"))
max_temp = float(input("What is the maximum temperature you would like for your trip (F)?"))

What is the minimum temperature you would like for your trip (F)?70
What is the maximum temperature you would like for your trip (F)?90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"]<=max_temp) & 
                                       (city_data_df["Max Temp"]>=min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Filiatra,GR,37.1500,21.5833,72.75,80,9,15.10,clear sky
3,3,Payo,ID,-3.7508,103.6372,73.72,96,100,1.97,moderate rain
6,6,Mecca,SA,21.4267,39.8261,87.24,43,1,2.44,clear sky
8,8,Mampong,GH,7.0627,-1.4001,81.66,67,81,2.75,light rain
9,9,Saint-Philippe,RE,-21.3585,55.7679,72.99,80,55,6.29,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                294
City                   294
Country                294
Lat                    294
Lng                    294
Max Temp               294
Humidity               294
Cloudiness             294
Wind Speed             294
Current Description    294
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Filiatra,GR,37.1500,21.5833,72.75,80,9,15.10,clear sky
3,3,Payo,ID,-3.7508,103.6372,73.72,96,100,1.97,moderate rain
6,6,Mecca,SA,21.4267,39.8261,87.24,43,1,2.44,clear sky
8,8,Mampong,GH,7.0627,-1.4001,81.66,67,81,2.75,light rain
9,9,Saint-Philippe,RE,-21.3585,55.7679,72.99,80,55,6.29,broken clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Filiatra,GR,72.75,clear sky,37.1500,21.5833,
3,Payo,ID,73.72,moderate rain,-3.7508,103.6372,
6,Mecca,SA,87.24,clear sky,21.4267,39.8261,
8,Mampong,GH,81.66,light rain,7.0627,-1.4001,
9,Saint-Philippe,RE,72.99,broken clouds,-21.3585,55.7679,
10,Saint George,US,88.54,clear sky,37.1041,-113.5841,
11,Albany,US,74.17,clear sky,42.6001,-73.9662,
12,Rikitea,PF,71.87,overcast clouds,-23.1203,-134.9692,
16,Labuhan,ID,79.70,broken clouds,-6.8844,112.2051,
17,Kavieng,PG,80.67,moderate rain,-2.5744,150.7967,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": gkey}

print("Retrieving Data")
print("---------------")

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]

    except:
        print("Hotel not found, skipping")

print("-----------------------")
print("Data Retrieval Compelte")
print("-----------------------")

Retrieving Data
---------------
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
Hotel not found, skipping
-----------------------
Data Retrieval Compelte
-----------------------


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Filiatra,GR,72.75,clear sky,37.1500,21.5833,Hotel Limenari
3,Payo,ID,73.72,moderate rain,-3.7508,103.6372,Mess keban agung
6,Mecca,SA,87.24,clear sky,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
8,Mampong,GH,81.66,light rain,7.0627,-1.4001,Adom Upscale Hotel
9,Saint-Philippe,RE,72.99,broken clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))